In [1]:
# coding=utf-8
"""
This module contains the tests which ensure that data validation checks always occur when running a bancor3-related arb_mode.
"""
from fastlane_bot import Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
import subprocess, os, sys
import pytest
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
SushiswapV2 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


# Setup

In [2]:
def find_main_py():
    # Start at the directory of the current script
    cwd = os.path.abspath(os.path.join(os.getcwd()))
    
    print(f"Searching for main.py in {cwd}")
    while True:
        # Check if main.py exists in the current directory
        if "main.py" in os.listdir(cwd):
            return cwd  # Found the directory containing main.py
        else:
            # If not, go up one directory
            new_cwd = os.path.dirname(cwd)

            # If we're already at the root directory, stop searching
            if new_cwd == cwd:
                raise FileNotFoundError("Could not find main.py in any parent directory")

            cwd = new_cwd
       
       
def run_command(arb_mode, expected_log_line):
    
    # Find the correct path to main.py
    main_script_path = find_main_py()
    print(f"Found main.py in {main_script_path}")
    main_script_path = main_script_path + "/main.py"

    # Run the command
    cmd = [
        "python",
        main_script_path,
        f"--arb_mode={arb_mode}",
        "--default_min_profit_gas_token=60",
        "--limit_bancor3_flashloan_tokens=False",
        "--loglevel=DEBUG",
        "--alchemy_max_block_fetch=5",
        "--logging_path=fastlane_bot/data/",
        "--timeout=120",
        "--blockchain=ethereum"
    ]
    subprocess.Popen(cmd)
        
    # Wait for the expected log line to appear
    found = False
    result = subprocess.run(cmd, text=True, capture_output=True, check=True, timeout=180)

    # Check if the expected log line is in the output
    if expected_log_line in result.stderr or expected_log_line in result.stdout:
        found = True

    if not found:
        pytest.fail("Expected log line was not found within 1 minute")  # If we reach this point, the test has failed

## Test Data Validation For b3_two_hop

In [3]:
expected_log_line = "Transactions will be required to pass data validation for"
arb_mode = "b3_two_hop"
run_command(arb_mode=arb_mode, expected_log_line=expected_log_line)

Searching for main.py in /Users/mikewcasale/Documents/GitHub/bancor/fastlane-bot/resources/NBTest
Found main.py in /Users/mikewcasale/Documents/GitHub/bancor/fastlane-bot


Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line

2023-12-10 15:13:17,387 [fastlane:DEBUG] - [events.managers.contracts._get_and_save_token_info_from_contract] Adding new token DS-5f94 to fastlane_bot/data/blockchain_data/ethereum/tokens.csv
2023-12-10 15:13:17,499 [fastlane:DEBUG] - [events.managers.contracts._get_and_save_token_info_from_contract] Adding new token GEM-efcC to fastlane_bot/data/blockchain_data/ethereum/tokens.csv
2023-12-10 15:13:18,314 [fastlane:DEBUG] - [events.managers.contracts._get_and_save_token_info_from_contract] Adding new token BabyDoge-460B to fastlane_bot/data/blockchain_data/ethereum/tokens.csv
2023-12-10 15:13:19,739 [fastlane:DEBUG] - [events.managers.contracts._get_and_save_token_info_from_contract] Adding new token CPD-5355 to fastlane_bot/data/blockchain_data/ethereum/tokens.csv
2023-12-10 15:13:20,393 [fastlane:DEBUG] - [events.managers.contracts._get_and_save_token_info_from_contract] Adding new token ML-4Cc6 to fastlane_bot/data/blockchain_data/ethereum/tokens.csv
2023-12-10 15:13:22,830 [fastlan

2023-12-10 15:14:51,333 [fastlane:DEBUG] - [modes.base._set_best_ops] *************
2023-12-10 15:14:51,333 [fastlane:DEBUG] - [modes.base._set_best_ops] New best profit: 1.023337244987489e-12
2023-12-10 15:14:51,335 [fastlane:DEBUG] - [modes.base._set_best_ops] best_trade_instructions_df:                                                         BNT-FF1C  ...      ENJ-3B9c
0x748ab2bef0d97e5a044268626e6c9c104bab818605d44...  1.862645e-09  ...           NaN
0xb05d72ef454cca7774af818b39b03092727554a3e2c2a...  1.862645e-09  ... -3.725290e-09
PRICE                                               2.134869e+00  ...  1.000000e+00
AMMIn                                               3.725290e-09  ...  0.000000e+00
AMMOut                                              0.000000e+00  ... -3.725290e-09
TOTAL NET                                           3.725290e-09  ... -3.725290e-09

[6 rows x 3 columns]
2023-12-10 15:14:51,335 [fastlane:DEBUG] - [modes.base.calculate_profit] *************
2023-12-10 1

2023-12-10 15:15:39,788 [fastlane:INFO] - [bot._run] No eligible arb opportunities.
2023-12-10 15:15:39,788 [fastlane:INFO] - 

********************************************
Average Total iteration time for loop 3: 43.259783347447716
********************************************


2023-12-10 15:15:51,789 [fastlane:INFO] - [main] Timeout hit... stopping bot
